### Démarrage de tensorboard et imports principaux

In [1]:
# Agrandir le notebook ?
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# Commande à entrer dans le prompt anaconda si on veut éviter que notre jupyter crash
#  sur un long entrainement
#jupyter notebook --NotebookApp.iopub_data_rate_limit=100000000

# Démarrage de tensorboard pour notebook
%load_ext tensorboard


import sys
from matplotlib import pyplot
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten, \
Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import datetime
import os
import pandas as pd
from PIL import Image
import shutil  
from math import ceil, floor

from tensorflow.python.framework import ops #pour tenter de reset tensorboard, sans grand succès
ops.reset_default_graph()

### Hyper paramètres


In [2]:
'''
#Real Test 115C -> variations du meilleur modèle acutellement trouvé avec différentes data augmentation

# CONSTANTES : nb_layers=  8, epochs = 100, filters_per_layers = 64, batch_size=100, lr à 0.001, momentum = 0.9, optimizer Adam, 
#  padding = same, avg_pool, relu, kernel = (3,3), filters_per_layer = 64 filters_double = 0,
#  batchnorm, l1 et l2 à 0.0001, dropout à 0.4, MLP 128,
#  pool_frequency = 2, pool_frequency_change = (0,0), filters_double = 0 (jamais)
#
#3 tests

#Meilleur modèle avec des datas aug différentes
#  * Petite data augmentation : variation sur la largeur et la hauteur de 0.1 et flip horizontal aléatoire
#  * Data augmentation un peu plus étoffée : pareil avec un zoom pouvant varier jusqu'à 30%
#  * Grosse Data Augmentation : pareil avec rotation pouvant aller jusq'à 45°



list_indiv_id = ['1', '2', '3']
list_epochs = [60, 60, 60]
list_batch_size = [100, 100, 100]
list_nb_layers = [8,8,8]
list_l1 = [0.0001, 0.0001, 0.0001]
list_l2 = [0.0001, 0.0001, 0.0001]
list_batch_norm = [1, 1, 1]
list_dropout = [0.4, 0.4, 0.4]
list_filters_per_layers = [64, 64, 64]
list_filters_double = [0, 0, 0]
list_MLP_end = [128, 128, 128]
list_activation = ['relu','relu','relu']
list_kernel = [(3,3),(3,3),(3,3)]
list_padding = ['same','same','same']
list_max_or_avg_pool = ['avg','avg','avg']
list_pool_frequency = [2, 2, 2]
list_pool_frequency_change = [(0,0),(0,0),(0,0)]
list_learning_r = [0.001,0.001,0.001]
list_momentum = [0.9,0.9,0.9]
list_optimizer = ['Adam','Adam','Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time
'''

'\n#Real Test 115C -> variations du meilleur modèle acutellement trouvé avec différentes data augmentation\n\n# CONSTANTES : nb_layers=  8, epochs = 100, filters_per_layers = 64, batch_size=100, lr à 0.001, momentum = 0.9, optimizer Adam, \n#  padding = same, avg_pool, relu, kernel = (3,3), filters_per_layer = 64 filters_double = 0,\n#  batchnorm, l1 et l2 à 0.0001, dropout à 0.4, MLP 128,\n#  pool_frequency = 2, pool_frequency_change = (0,0), filters_double = 0 (jamais)\n#\n#3 tests\n\n#Meilleur modèle avec des datas aug différentes\n#  * Petite data augmentation : variation sur la largeur et la hauteur de 0.1 et flip horizontal aléatoire\n#  * Data augmentation un peu plus étoffée : pareil avec un zoom pouvant varier jusqu\'à 30%\n#  * Grosse Data Augmentation : pareil avec rotation pouvant aller jusq\'à 45°\n\n\n\nlist_indiv_id = [\'1\', \'2\', \'3\']\nlist_epochs = [60, 60, 60]\nlist_batch_size = [100, 100, 100]\nlist_nb_layers = [8,8,8]\nlist_l1 = [0.0001, 0.0001, 0.0001]\nlist_l2

In [3]:
#Real Test 15C FINAL -> meilleur modèle + faible data aug sur 200 epochs

# CONSTANTES : epochs = 200, batch_size = 100, nb_layers = 8,
# filters_per_layers à 64, pas de filters_double, MLP à 128,
# activation relu, kernel = (3,3), padding = same, avg pool, pool_frequency = 2,
# pool_frequency_change à (0,0) (pas activée), learning_r à 0.001, momentum à 0.9, optimizer Adam
#
#1 test

#Meilleur modèle avec petite data augmentation : variation sur la largeur et la hauteur de 0.1 et flip horizontal aléatoire

list_indiv_id = ['1']
list_epochs = [200]
list_batch_size = [100]
list_nb_layers = [8]
list_l1 = [0.0001]
list_l2 = [0.0001]
list_batch_norm = [1]
list_dropout = [0.4]
list_filters_per_layers = [64]
list_filters_double = [0]
list_MLP_end = [128]
list_activation = ['relu']
list_kernel = [(3,3)]
list_padding = ['same']
list_max_or_avg_pool = ['avg']
list_pool_frequency = [2]
list_pool_frequency_change = [(0,0)]
list_learning_r = [0.001]
list_momentum = [0.9]
list_optimizer = ['Adam']

current_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
main_directory = os.getcwd() + "\\logs\\convnets\\logs_" + current_time

### Fonctions pour préparer le dataset

In [4]:
# Chargement des datasets de train et de validation + one hot encoding
def load_dataset():
    # Chargement des données cifar10
    (trainX, trainY), (testX, testY) = cifar10.load_data()
    # one hot encode encoding sur les labels
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# Normalisation pour accroître la vitesse du modèle (en redimensionnant les pixels)
def prep_pixels(train, test):
    # Convertion des int en float
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # Normalisation pour avoir des nombres entre 0 et 1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # Retourner les images normalisées
    return train_norm, test_norm


### Classe Python pour définir les individus

In [5]:
# Classe pour les convnets
class IndividuConvnets:
    def __init__(self, indiv_id='1', epochs=10, batch_size=1, nb_layers=2, l1=0, l2=0, batch_norm=0,
                 dropout=0, filters_per_layers=64, filters_double=6, MLP_end=0, activation='relu',
                 kernel=(3,3), padding='same', max_or_avg_pool=0, pool_frequency=2,
                 pool_frequency_change = (0,0), learning_r=0.01, momentum=0.9, optimizer='SGD'):
        
        # Initialisation de nos variables
        self.time_fit = datetime.datetime.now()
        self.my_reguralizer = None
        
        
        self.nb_layers = nb_layers
            
        self.loss = 0
        self.accuracy = 0
        self.indiv_id = indiv_id
        self.epochs = epochs
        self.batch_size = batch_size

        self.l1 = l1
        self.l2 = l2

        self.batch_norm = batch_norm
        self.dropout = dropout
        self.filters_per_layers = filters_per_layers
                
        self.filters_double = filters_double
        
        if MLP_end < 0:
            self.MLP_end = 0
        else:
            self.MLP_end = MLP_end
        
        self.activation = activation
        self.kernel = kernel
        self.padding = padding
        self.max_or_avg_pool = max_or_avg_pool
        self.pool_frequency = pool_frequency
        self.pool_frequency_change = pool_frequency_change
        self.learning_r = learning_r
        self.momentum = momentum
        self.optimizer = optimizer
    
    # ToString()
    def __str__(self):
        ma_liste = []
        ma_liste.append("indiv_id:{},\n ".format(self.indiv_id))
        ma_liste.append("epochs:{},\n ".format(self.epochs))
        ma_liste.append("batch_size:{},\n ".format(self.batch_size))
        ma_liste.append("nb_layers:{},\n ".format(self.nb_layers))
        ma_liste.append("l1:{},\n ".format(self.l1))
        ma_liste.append("l2:{},\n ".format(self.l2))
        ma_liste.append("batch_norm:{},\n ".format(self.batch_norm))
        ma_liste.append("dropout:{},\n ".format(self.dropout))
        ma_liste.append("filters_per_layers:{},\n ".format(self.filters_per_layers))
        ma_liste.append("filters_double:{},\n ".format(self.filters_double))
        ma_liste.append("MLP_end:{},\n ".format(self.MLP_end))
        ma_liste.append("activation:{},\n ".format(self.activation))
        ma_liste.append("kernel:\n ")
        ma_liste.append("{},\n ".format(self.kernel))
        ma_liste.append("padding:{},\n ".format(self.padding))
        ma_liste.append("max_or_avg_pool:{}\n".format(self.max_or_avg_pool))
        ma_liste.append("pool_frequency:{}\n".format(self.pool_frequency))
        ma_liste.append("pool_frequency_change:{}\n".format(self.pool_frequency_change))
        ma_liste.append("learning_r:{}\n".format(self.learning_r))
        ma_liste.append("momentum:{}\n".format(self.momentum))
        ma_liste.append("optimizer:{}\n".format(self.optimizer))
            
        return ma_liste
    
    # (Modele 2 conv + norm ? + pool) * X -> MLP -> softmax sortie 10 -> MODELE BLOC 2
    # D'autres modeles seront crees par la suite
    def create_and_train_model(self, trainX, trainY, testX, testY, main_directory):
        start = datetime.datetime.now()
        
        # Choix d'un emplacement pour les logs
        log_dir=main_directory+"\\log_"+self.indiv_id+"\\tensorboard_data\\"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        print("log dir = ",log_dir)
        
        # l1 et l2
        if self.l1 > 0 and self.l2 > 0:
            self.my_regularizer = regularizers.l1_l2(l1=self.l1 / self.nb_layers,
                                    l2=self.l2 / self.nb_layers)
        elif self.l1 > 0:
            self.my_regularizer = regularizers.l1(self.l1 / self.nb_layers)
        elif self.l2 > 0:
            self.my_regularizer = regularizers.l2(self.l2 / self.nb_layers)
        else:
            self.my_reguralizer = None
            
        # Definir notre modèle basique
        model = Sequential()

        # Faire toutes les convs nécessaires
        counter_filters_double = 0 # Var pour doubler les filtres
        counter_pool_freq = 0 # var pour savoir où placer les couches de pooling
        counter_pool = 0 # var pour nommer les max / avg pool
        
        #initialisation de variables locales pour ne pas modifier nos attributs evolutifs
        # qui doivent être log à la fin
        pool_frequency = self.pool_frequency
        filters_per_layers = self.filters_per_layers
        
        for i in range(0, self.nb_layers):
            
            print("counter_pool_freq = ", counter_pool_freq)
            # Traitement pour doubler les filtres tous les X couches de convo
            if counter_filters_double >= self.filters_double and self.filters_double > 0:
                filters_per_layers = filters_per_layers * 2
                print("filters = ", filters_per_layers)
                counter_filters_double = 0
            
            # Première conv, on fixe l'input shape
            if i == 0:
                model.add(Conv2D(filters_per_layers, self.kernel, activation=self.activation,
                    kernel_regularizer=self.my_reguralizer, padding=self.padding,
                    input_shape=(32, 32, 3), name='conv_'+str(filters_per_layers)+'_'+str(i+1)))
            else:
                # Couche de conv + rajouts selon nos hyperparams
                model.add(Conv2D(filters_per_layers, self.kernel, activation=self.activation,
                    kernel_regularizer=self.my_reguralizer, padding=self.padding,
                                 name='conv_'+str(filters_per_layers)+'_'+str(i+1)))
            
            # Après avoir créé une conv on incrémente nos compteurs (sauf counter_pool)
            counter_filters_double = counter_filters_double + 1
            counter_pool_freq = counter_pool_freq + 1
            
            # Ajouts de la regularization / du pooling selon les hyperparamètres saisis
            
            if self.batch_norm == 1:
                model.add(BatchNormalization( name='batchnorm_'+str(i+1)))
            
            if pool_frequency == counter_pool_freq:    
                #go max ou avg pooling
                if self.max_or_avg_pool == 'max':
                    model.add(MaxPooling2D((2, 2), padding='same', 
                        name='max_pool_'+str(counter_pool+1)))
                    counter_pool = counter_pool + 1
                else:
                    model.add(AveragePooling2D((2, 2), padding='same', 
                                name='avg_pool_'+str(counter_pool+1)))
                    counter_pool = counter_pool + 1
                
                # Dropout sur les pools 
                if self.dropout > 0:
                    model.add(Dropout(self.dropout, 
                                      name='Dropout_'+str(self.dropout)+'_'+str(counter_pool+1)))
                    
                # si filters_double à -1, on double les filtres apres le pooling
                if self.filters_double == -1:
                    filters_per_layers = filters_per_layers * 2
                
                # Après avoir mis un pool, on regarde si l'on doit changer ou non 
                #  la freq d'apparition de pools
                if counter_pool == self.pool_frequency_change[0] \
                    and self.pool_frequency_change[0] != 0:
                    
                    pool_frequency = pool_frequency + self.pool_frequency_change[1]
                    
                counter_pool_freq = 0
            
        
        # Fin des convs -> neural network classique
        model.add(Flatten(name='Flatten'))
        
        print("i after for = ", i)
        #tTrain dans un MLP avant la fin si on le souhaite
        if self.MLP_end > 0:
            model.add(Dense(128, activation='relu', kernel_regularizer=self.my_reguralizer,
                            name='MLP_'+str(self.MLP_end)))
            if self.batch_norm == 1:
                model.add(BatchNormalization( name='batchnorm_finale'))
            
            #mettre dropout sur les Dense, moins opti sur des pools ? à tester si tmp ok
            #(pas conv car importantes)
            if self.dropout > 0:
                model.add(Dropout(self.dropout, name='Dropout_'+str(self.dropout)+'_final'))
        
        #notre output
        model.add(Dense(10, activation='softmax', name='output')) 

        # Compiler le modele
        if self.optimizer == 'SGD':
            print("SGD, learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = SGD(lr=self.learning_r, momentum=self.momentum)
        else:
            print("Adam learning_r = ", self.learning_r, " momentum = ", self.momentum, "\n")
            opt = Adam(lr=self.learning_r, beta_1=self.momentum) # beta_1 => notation momentum Adam
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
        # APPLICATION DE LA DATA AUGMENTATION
        # On crée un générateur d'images (avec peu, pas mal ou beaucoup de variations)
        if self.indiv_id == '1':
            datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
            print("Simple data aug")
        elif self.indiv_id == '2':
            datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, zoom_range=0.3)
            print("Moyenne data aug")
        else:
            datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True, zoom_range=0.3, rotation_range=45)
            print("Grosse data aug")
        # On crée un "iterator"
        it_train = datagen.flow(trainX, trainY, batch_size=self.batch_size)
        #On va devoir ajouter l'attribut steps_per_epoch au fit, il donne le nombre d'expemples de data aug présents dans chaque epoch
        steps = int(trainX.shape[0] / self.batch_size)
        
        # Entrainer le modele
        history = model.fit_generator(it_train, steps_per_epoch=steps, epochs=self.epochs,
                        validation_data=(testX, testY), verbose=1, callbacks=[tensorboard_callback])
        
        # Garder une trace du temps nécessaire pour fit (peut être pas la meilleure méthode)
        end = datetime.datetime.now()
        self.time_fit = end - start
        print("\nTime for fit = ", round(self.time_fit.total_seconds(),2)) # Round total_seconds()

        return history, model
    
    
    def save_model(self, history, model, main_directory, current_time):
        
        # Sauvegarde du modèle
        plot_model(model, "model.png")
        
        # Deplacement modele au bon endroit
        shutil.move(os.getcwd()+"\\model.png", main_directory+"\\log_"+self.indiv_id+"\\model.png")
        
        # Afficher nos résultats dans un graphique matplotlib sauvegardé
        pyplot.gcf().subplots_adjust(hspace = 0.5)

        # Afficher la loss
        pyplot.subplot(211)
        pyplot.title('Cross Entropy Loss')
        pyplot.plot(history.history['loss'], color='blue', label='train')
        pyplot.plot(history.history['val_loss'], color='orange', label='test')
        
        # Afficher l'accuracy
        pyplot.subplot(212)
        pyplot.title('Classification Accuracy')
        pyplot.plot(history.history['accuracy'], color='blue', label='train')
        pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
        
        # Sauvegarde
        filename = main_directory+"\\log_"+self.indiv_id+"\\"
        pyplot.savefig(filename + 'plot.png')
        pyplot.close()
       
        
        print("LOSS : ", round(history.history['loss'][self.epochs-1].item(), 3))
        print("VAL_LOSS : ", round(history.history['val_loss'][self.epochs-1].item(), 3))
        print("ACCURACY : ", round(history.history['accuracy'][self.epochs-1].item(), 3))
        print("VAL_ACCURACY : ", round(history.history['val_accuracy'][self.epochs-1].item(), 3))
        
        # attributs pour créer les csv indivudels et le csv global
        self.loss = round(history.history['loss'][self.epochs-1].item(), 3)
        self.val_loss = round(history.history['val_loss'][self.epochs-1].item(), 3)
        self.accuracy = round(history.history['accuracy'][self.epochs-1].item(), 3)
        self.val_accuracy = round(history.history['val_accuracy'][self.epochs-1].item(), 3)
        self.time_taken = round(self.time_fit.total_seconds(),2)
        
        # Créer un dataframe pandas (avec hyperparams) et le sauvegarder en CSV
        df = pd.DataFrame({'indiv_id': [self.indiv_id],
                           'epochs': [self.epochs],
                           'batch_size': [self.batch_size],
                           'nb_layers': [self.nb_layers],
                           'l1': [self.l1],
                           'l2': [self.l2],
                           'batch_norm': [self.batch_norm],
                           'dropout': [self.dropout],
                           'filters_per_layers': [self.filters_per_layers],
                           'filters_double': [self.filters_double],
                           'MLP_end': [self.MLP_end],
                           'activation': [self.activation],
                           'kernel': [self.kernel],
                           'padding': [self.padding],
                           'max_or_avg_pool': [self.max_or_avg_pool],
                           'pool_frequency': [self.pool_frequency],
                           'pool_frequency_change': [self.pool_frequency_change],
                           'loss': [self.loss],
                           'val_loss': [self.val_loss],
                           'accuracy': [self.accuracy],
                           'val_accuracy': [self.val_accuracy],
                           'time_taken' : [self.time_taken],
                           'learning_r' : [self.learning_r],
                           'momentum' : [self.momentum],
                           'optimizer' : [self.optimizer]
                          })
        
        df.to_csv(path_or_buf=filename+"recap.csv",index=False)
    
    # Lance toutes les étapes
    def exec_indiv(self, main_directory, current_time):
        
        # Charger les données
        trainX, trainY, testX, testY = load_dataset()
        
        # Normaliser les données
        trainX, testX = prep_pixels(trainX, testX)
        
        print("TrainX shape = ",np.shape(trainX))
        print("TestX shape = ",np.shape(testX), "\n")
        # Créer et entrainer le modele
        history, model = self.create_and_train_model(trainX, trainY, testX, testY, main_directory)
        
        # Sauvegarder le modèle
        save = self.save_model(history, model, main_directory, current_time)

### Classe Python qui va démarrer les tests des neural nets


In [6]:
# Classe générale qui va nous servir à effectuer des actions sur des individus
class MyTraining:
    # Prends un ID et une liste d'individus 
    def __init__(self, id_train, indiv_list):
        
        self.id_train = id_train
        self.indiv_list = indiv_list
    
    def train(self, main_directory, current_time):
        
        print("Start training\n")
        
        for indiv in self.indiv_list:
            print("indiv ", indiv.indiv_id, "\n")
            indiv.exec_indiv(main_directory, current_time)
            print("-----------------------------------------------------------------\n")
        
        # Fusion des csv 
        merge_csv = pd.DataFrame(columns=['indiv_id', 'epochs', 'nb_layers', 'l1', 'l2', 'batch_norm', 'dropout',
                                          'filters_per_layers', 'filters_double', 'MLP_end', 'activation', 'kernel',
                                          'padding','max_or_avg_pool', 'pool_frequency', 'pool_frequency_change', 'loss',
                                          'val_loss', 'accuracy', 'val_accuracy', 'time_taken','learning_r',
                                          'momentum', 'optimizer'])
        
        for indiv in self.indiv_list:
            merge_csv = merge_csv.append(
                             {'indiv_id': indiv.indiv_id, 'epochs': indiv.epochs, 'batch_size': indiv.batch_size,
                              'nb_layers' : indiv.nb_layers,'l1' : indiv.l1, 'l2' : indiv.l2, 'batch_norm': indiv.batch_norm,
                              'dropout' : indiv.dropout,'filters_per_layers' : indiv.filters_per_layers,
                              'filters_double' : indiv.filters_double,'MLP_end' : indiv.MLP_end,
                              'activation' : indiv.activation,'kernel' : indiv.kernel,'padding' : indiv.padding,
                              'max_or_avg_pool' : indiv.max_or_avg_pool,'pool_frequency' : indiv.pool_frequency,
                              'pool_frequency_change' : indiv.pool_frequency_change,'loss' : indiv.loss,
                              'val_loss' : indiv.val_loss,'accuracy' : indiv.accuracy, 'val_accuracy' : indiv.val_accuracy,
                              'time_taken' : indiv.time_taken,'learning_r' : indiv.learning_r,'momentum': indiv.momentum,
                              'optimizer' : indiv.optimizer
                             },ignore_index=True)
        
        # sauvegarde
        merge_csv.to_csv(main_directory+"\\combined_recap.csv", index=False)
            
    
    def all_indiv(self):
        
        # Affiche les caractéristiques de l'ensemble des individus
        for indiv in self.indiv_list:
            print('\n'.join(indiv.__str__()))
            for tir in range(80): print('-', end='')
            print()

### Traitement général (train de l'ensemble des modèles)

In [7]:
# Création des individus (des neurals nets, ici convnet)
list_indiv = []
for num in range(len(list_indiv_id)):
    list_indiv.append(IndividuConvnets(
        list_indiv_id[num],
          list_epochs[num],
          list_batch_size[num],
          list_nb_layers[num],
          list_l1[num],
          list_l2[num],
          list_batch_norm[num],
          list_dropout[num],
          list_filters_per_layers[num],
          list_filters_double[num],
          list_MLP_end[num],
          list_activation[num],
          list_kernel[num],
          list_padding[num],
          list_max_or_avg_pool[num],
          list_pool_frequency[num],
          list_pool_frequency_change[num],
          list_learning_r[num],
          list_momentum[num],
          list_optimizer[num]
        )
    )

    
# Chargement de la classe training, affichag
training_1 = MyTraining(1, list_indiv)
training_1.all_indiv()
training_1.train(main_directory, current_time)

indiv_id:1,
 
epochs:200,
 
batch_size:100,
 
nb_layers:8,
 
l1:0.0001,
 
l2:0.0001,
 
batch_norm:1,
 
dropout:0.4,
 
filters_per_layers:64,
 
filters_double:0,
 
MLP_end:128,
 
activation:relu,
 
kernel:
 
(3, 3),
 
padding:same,
 
max_or_avg_pool:avg

pool_frequency:2

pool_frequency_change:(0, 0)

learning_r:0.001

momentum:0.9

optimizer:Adam

--------------------------------------------------------------------------------
Start training

indiv  1 

TrainX shape =  (50000, 32, 32, 3)
TestX shape =  (10000, 32, 32, 3) 

log dir =  C:\Users\arnau\Desktop\quatrième_année\Deep_Learning\Projet_cifar-10\logs\convnets\logs_2020-02-10-00-18\log_1\tensorboard_data\
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
counter_pool_freq =  0
counter_pool_freq =  1
i after for =  7
Adam learning_r =  0.001  momentum =  0.9 

Simple data aug
Epoch 1/200
500/500 [==============================] - 81s 162ms/step 

500/500 [==============================] - 153s 305ms/step - loss: 0.4333 - accuracy: 0.8550 - val_loss: 0.5061 - val_accuracy: 0.8410
Epoch 51/200
500/500 [==============================] - 151s 302ms/step - loss: 0.4266 - accuracy: 0.8575 - val_loss: 0.4274 - val_accuracy: 0.8556
Epoch 52/200
500/500 [==============================] - 154s 308ms/step - loss: 0.4269 - accuracy: 0.8556 - val_loss: 0.5610 - val_accuracy: 0.8256
Epoch 53/200
500/500 [==============================] - 149s 297ms/step - loss: 0.4265 - accuracy: 0.8571 - val_loss: 0.4405 - val_accuracy: 0.8539
Epoch 54/200
500/500 [==============================] - 152s 304ms/step - loss: 0.4218 - accuracy: 0.8595 - val_loss: 0.4851 - val_accuracy: 0.8435
Epoch 55/200
500/500 [==============================] - 170s 340ms/step - loss: 0.4178 - accuracy: 0.8594 - val_loss: 0.4959 - val_accuracy: 0.8403
Epoch 56/200
500/500 [==============================] - 183s 365ms/step - loss: 0.4168 - accuracy: 0.8605 - val_loss: 0.4861 

500/500 [==============================] - 149s 297ms/step - loss: 0.3541 - accuracy: 0.8796 - val_loss: 0.3868 - val_accuracy: 0.8768
Epoch 106/200
500/500 [==============================] - 150s 301ms/step - loss: 0.3565 - accuracy: 0.8788 - val_loss: 0.5184 - val_accuracy: 0.8403
Epoch 107/200
500/500 [==============================] - 153s 306ms/step - loss: 0.3500 - accuracy: 0.8817 - val_loss: 0.3954 - val_accuracy: 0.8750
Epoch 108/200
500/500 [==============================] - 148s 296ms/step - loss: 0.3507 - accuracy: 0.8809 - val_loss: 0.4757 - val_accuracy: 0.8495
Epoch 109/200
500/500 [==============================] - 147s 294ms/step - loss: 0.3483 - accuracy: 0.8840 - val_loss: 0.4325 - val_accuracy: 0.8641
Epoch 110/200
500/500 [==============================] - 158s 316ms/step - loss: 0.3446 - accuracy: 0.8832 - val_loss: 0.3852 - val_accuracy: 0.8783
Epoch 111/200
500/500 [==============================] - 146s 292ms/step - loss: 0.3484 - accuracy: 0.8816 - val_loss: 0

500/500 [==============================] - 203s 406ms/step - loss: 0.3175 - accuracy: 0.8929 - val_loss: 0.4731 - val_accuracy: 0.8560
Epoch 161/200
500/500 [==============================] - 159s 319ms/step - loss: 0.3214 - accuracy: 0.8905 - val_loss: 0.3477 - val_accuracy: 0.8876
Epoch 162/200
500/500 [==============================] - 163s 326ms/step - loss: 0.3135 - accuracy: 0.8946 - val_loss: 0.3734 - val_accuracy: 0.8825
Epoch 163/200
500/500 [==============================] - 154s 308ms/step - loss: 0.3203 - accuracy: 0.8919 - val_loss: 0.3700 - val_accuracy: 0.8808
Epoch 164/200
500/500 [==============================] - 156s 312ms/step - loss: 0.3198 - accuracy: 0.8902 - val_loss: 0.3582 - val_accuracy: 0.8851
Epoch 165/200
500/500 [==============================] - 167s 334ms/step - loss: 0.3124 - accuracy: 0.8944 - val_loss: 0.4050 - val_accuracy: 0.8739
Epoch 166/200
500/500 [==============================] - 175s 350ms/step - loss: 0.3149 - accuracy: 0.8915 - val_loss: 0

### Partie tensorboard

In [13]:
# Procedure pour utiliser tensorboard
#  1 load la première cell
#  2 utiliser la derniere cell avec --logdir (précisez bien votre répertoire, plus sur que ça
#    fonctionne avec une string "mon_path"
#  3 Vous NE POURREZ PLUS update tensorboard sur ce port et il y aura des bugs, pour éviter ça
#    quand vous voulez faire une update, fermez jupyter notebook (shutdown total) et réouvrez le 
#    OU, faites kernel->interrupt et changez de port + de folder de log

#si vous voulez tenter de tuer des process
#os.system("taskkill /im tensorboard.exe /f") #kill tous les processus qui utilisent tensorboard
#os.system('!kill 18776') #kill le processus X

In [ ]:
# Liste des ports utilisés par tensorboard, attention ça se remplit vite et il faut kill jupyter pour clean
from tensorboard import notebook
notebook.list()

In [ ]:
# Code pour démarrer tensorboard dans le dossier souhaité [PRECISEZ BIEN LE DOSSIER ICI]
%tensorboard --logdir "CONVNETS_20200119-0243\logs_20200119-093909\tensorboard_data" --port=6066

In [ ]:
# Si vous avez la folie des grandeurs
notebook.display(port=6066, height=1000) 

In [ ]:
### Fichier CSV combined_recap + Graphique

In [ ]:
# Commandes pandas utiles
data_csv = pd.read_csv(main_directory + "\\combined_recap.csv")
#data_csv = pd.read_csv("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\CONVNETS_20200119-2043\\combined_recap.csv")
data.head()
#meilleure accuracy, moins pire loss par ex
#data_csv.sort_values(["elapsed"], axis=0, 
                 #ascending=[False], inplace=True) 

# Afficher uniquement certaines colonnes
#dataX = data_csv.filter(items=['elapsed', 'label'])

#récupérer uniquement où la loss est < à X et ou kernel = (3,3) par exemple
#dataX = data_csv.loc[(data_csv['elapsed'] > 700) & (data_csv['threadName'].str.contains('Thread Group 1-2'))]
#dataX

#pd.set_option('display.max_rows', data3.shape[0]+1) #nombre de row max à afficher
#data_csv = pd.read_csv(main_directory+"\\logs_20200116-204456\\recap.csv")
#data_csv.head()

In [ ]:
image = pyplot.imread(main_directory + "\\logs_20200119-093909\\plot.png")
#image = pyplot.imread("C:\\Users\\arnau\\Desktop\\quatrième_année\\Deep_Learning\\Projet_cifar-10\\logs_20200119-093909\\plot.png")
pyplot.imshow(image)